# Google Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
% cd '/content/drive/My Drive/Pytorch Learning/Python3'

/content/drive/My Drive/Pytorch Learning/Python3


In [3]:
! ls

CIFAR10.py	     LeNet5.ckpt      __pycache__
LeNet5_CIFAR10.ckpt  LeNet5_MNIST.py  RNN_sentiemnt_custom_dataset_GloVe.ipynb
LeNet5_CIFAR10.py    LeNet5.py	      RNN_sentiemnt_custom_dataset.ipynb


# Code

In [4]:
# coding=utf8
"""
@author: Yantong Lai
@date: 2019.12.4
"""

import torchtext
from torchtext.data import get_tokenizer
from torchtext import data, datasets

import torch
import torch.nn as nn
import torch.optim as optim

import random

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Dataset path
dataset_path = "../../data/aclImdb/"

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


####################################
#         Hyper-parameters         #
####################################
BATCH_SIZE = 256
LEARNING_RATE = 1e-3


####################################
#          Preparing Data          #
####################################
# 1. data.Field()
TEXT = data.Field(tokenize='spacy', include_lengths=True)
LABELS = data.LabelField()

# 2. data.TabularDataset
train_data, test_data = data.TabularDataset.splits(path=dataset_path,
                                                   train="train.tsv",
                                                   test="test.tsv",
                                                   fields=[('labels', LABELS), ('text', TEXT)],
                                                   format="tsv")

# train_data, test_data = datasets.IMDB.splits(TEXT, LABELS)

print("Number of train_data = {}".format(len(train_data)))
print("Number of test_data = {}".format(len(test_data)))

print("vars(train_data[0]) = {}\n".format(vars(train_data[0])))

# 3. Split train_data to train_data, valid_data
train_data, valid_data = train_data.split(random_state=random.seed(SEED))
print("Number of train_data = {}".format(len(train_data)))
print("Number of valid_data = {}".format(len(valid_data)))
print("Number of test_data = {}\n".format(len(test_data)))


# 4. data.BucketIterator
train_iter, valid_iter, test_iter = data.BucketIterator.splits((train_data, valid_data, test_data),
                                                               batch_size=BATCH_SIZE,
                                                               device=device,
                                                               sort_key=lambda x: len(x.text))
# 5. Build vocab
# TEXT.build_vocab(train_data)
# unk_init=torch.Tensor.normal_)
# LABELS.build_vocab(train_data)
# print("vars(train_data[0]) = ", vars(train_data[0]))

# 5.1 (Optional) If build vocab with pre-trained word embedding vectors
TEXT.build_vocab(train_data, vectors="glove.6B.100d")
LABELS.build_vocab(train_data)
print("vars(train_data[0]) = ", vars(train_data[0]))



####################################
#          Build the Model         #
####################################
class RNN(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super(RNN, self).__init__()

        # 1. Embedding
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # 2. RNN layer
        self.rnn = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional,
                           dropout=dropout)

        # 3. Linear layer
        self.fc = nn.Linear(in_features=hidden_dim * 2,
                            out_features=output_dim)

    def forward(self, text, text_lengths):

        # 1. Embedding
        # text = [sent len, batch size]
        embedded = self.embedding(text)

        # 2. Pack sequence
        # embedded = [sent len, batch size, embed size]
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths, enforce_sorted=False)

        # 3. RNN
        packed_output, (hidden, cell) = self.rnn(packed_embedded)

        # 4. Unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        # output = [sent len, batch size, hid dim * num_directions]
        # output over padding tokens are zero tensors

        # hidden = [num_layers * num_directions, batch size, hid dim]
        # cell = [num_layers * num_directions, batch_size, hid dim]

        # 5. Concat the final forward (hidden[-2, :, :]) and backward (hidden[-1, :, :])
        hidden = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        # hidden = [batch size, hid dim * num_directions]

        # return self.fc(hidden.squeeze(0)).view(-1)
        return self.fc(hidden)

# Parameters
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 2
N_LAYERS = 2
BIDIRECTIONAL = True
DROUPOUT = 0.5
# PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

print("INPUT_DIM = {}".format(INPUT_DIM))
print("OUTPUT_DIM = {}".format(OUTPUT_DIM))
# print("TEXT.pad_token = {}".format(TEXT.pad_token))
# print("PAD_IDX = {}".format(PAD_IDX))

# Create an RNN instance
model = RNN(vocab_size=INPUT_DIM,
            embedding_dim=EMBEDDING_DIM,
            hidden_dim=HIDDEN_DIM,
            output_dim=OUTPUT_DIM,
            n_layers=N_LAYERS,
            bidirectional=BIDIRECTIONAL,
            dropout=DROUPOUT)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print("The model has {} trainable parameters".format(count_parameters(model)))


####################################
#          Train the Model         #
####################################
# Loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

model = model.to(device)


########## Train ##########
NUM_EPOCHS = 10
total_step = len(train_iter)
for epoch in range(NUM_EPOCHS):
    total_loss = []
    for i, batch in enumerate(train_iter):

        text, text_lengths = batch.text
        y = batch.labels
        # print("y = {}".format(y))

        # Forward pass
        y_pred = model(text, text_lengths)
        # print("y_pred = {}".format(y_pred))
        loss = criterion(y_pred, y)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss.append(loss.item())

        if (i + 1) % 10 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch + 1, NUM_EPOCHS, i + 1, total_step, loss.item()))

    print("total_loss = {}\n".format(total_loss))


########## Test ##########
model.eval()
total_correct = 0
avg_loss = 0.0
for i, batch in enumerate(test_iter):
    text, text_lengths = batch.text

    y = batch.labels

    # Forward pass
    y_pred = model(text, text_lengths)
    loss = criterion(y_pred, y)
    avg_loss += loss.item()

    # _, pred = torch.max(output.data, 1)
    pred = torch.argmax(y_pred.data, dim=1)
    total_correct += (pred == y).sum().item()

avg_loss = avg_loss / len(test_data)
print("Test Avg. Loss: {}, Accuracy: {}%"
      .format(avg_loss, 100 * total_correct / len(test_data)))


Number of train_data = 25000
Number of test_data = 25000
vars(train_data[0]) = {'labels': 'pos', 'text': ['Bromwell', 'High', 'is', 'a', 'cartoon', 'comedy', '.', 'It', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life', ',', 'such', 'as', '"', 'Teachers', '"', '.', 'My', '35', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'Bromwell', 'High', "'s", 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', '"', 'Teachers', '"', '.', 'The', 'scramble', 'to', 'survive', 'financially', ',', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', 'teachers', "'", 'pomp', ',', 'the', 'pettiness', 'of', 'the', 'whole', 'situation', ',', 'all', 'remind', 'me', 'of', 'the', 'schools', 'I', 'knew', 'and', 'their', 'students', '.', 'When', 'I', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school', ',', 

.vector_cache/glove.6B.zip: 862MB [06:31, 2.20MB/s]                           
100%|█████████▉| 398937/400000 [00:15<00:00, 26621.50it/s]

vars(train_data[0]) =  {'labels': 'neg', 'text': ['This', 'movie', 'has', 'got', 'to', 'be', 'one', 'of', 'the', 'worst', 'I', 'have', 'ever', 'seen', 'make', 'it', 'to', 'DVD', '!', '!', '!', 'The', 'story', 'line', 'might', 'have', 'clicked', 'if', 'the', 'film', 'had', 'more', 'funding', 'and', 'writers', 'that', 'would', 'have', 'cut', 'the', 'nonsense', 'and', 'sickly', 'scenes', 'that', 'I', 'highly', 'caution', 'parents', 'on', '....', 'But', 'the', 'story', 'line', 'is', 'like', 'a', 'loose', 'cannon', '.', 'If', 'there', 'was', 'such', 'a', 'thing', 'as', 'a', 'drive', 'thru', 'movie', 'maker', '-', 'this', 'one', 'would', 'have', 'sprung', 'from', 'that', '.', 'It', 'reminded', 'me', 'a', 'lot', 'of', 'the', 'quickie', 'films', 'that', 'were', 'put', 'out', 'in', 'the', '1960', "'s", ',', 'poor', 'script', 'writing', 'and', 'filming', '.', '<', 'br', '/><br', '/>The', 'only', 'sensible', 'characters', 'in', 'the', 'whole', 'movie', 'was', 'the', 'bartender', 'and', 'beaver', 

100%|█████████▉| 398937/400000 [00:29<00:00, 26621.50it/s]

Epoch [1/10], Step [10/69], Loss: 0.6829
Epoch [1/10], Step [20/69], Loss: 0.6739
Epoch [1/10], Step [30/69], Loss: 0.6387
Epoch [1/10], Step [40/69], Loss: 0.6598
Epoch [1/10], Step [50/69], Loss: 0.6832
Epoch [1/10], Step [60/69], Loss: 0.6102
total_loss = [0.6938649415969849, 0.6905660629272461, 0.6941353678703308, 0.689378559589386, 0.6933417916297913, 0.6792043447494507, 0.695667028427124, 0.6693956851959229, 0.697924792766571, 0.6829426288604736, 0.6896958947181702, 0.6787241697311401, 0.6732372641563416, 0.672018826007843, 0.6558898091316223, 0.6705135107040405, 0.6711212992668152, 0.6775741577148438, 0.680449903011322, 0.6739197373390198, 0.6648499369621277, 0.6462147235870361, 0.6752798557281494, 0.6568111181259155, 0.7018011212348938, 0.6360228657722473, 0.6259579658508301, 0.6817304491996765, 0.6352450847625732, 0.6387157440185547, 0.650404155254364, 0.6776708960533142, 0.672187864780426, 0.6507013440132141, 0.6595985293388367, 0.65294349193573, 0.6530744433403015, 0.6256859